In [1]:
import os
import copy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing, make_regression
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  

X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size=0.98, random_state=2)
XR, yR = make_regression(n_samples=100, n_features=2, noise=0, random_state=5)
X_train2, X_test2, y_train2, y_test2 = train_test_split(XR, yR, test_size=0.20, random_state=2)

def normalize_dataset(X):
    return tf.keras.utils.normalize(X)


c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [28]:
class Layer(tf.Module):
    def __init__(self, neurons=30, activation=tf.identity):
        self.neurons = neurons
        self.activation = activation
        self.build = False
        self.normalized = False

    @tf.function(reduce_retracing=True)
    def xavier_init(self, shape):
        in_dim, out_dim = shape
        xavier_lim = tf.sqrt(6.)/tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
        weight_vals = tf.cast(tf.random.uniform(shape=(in_dim, out_dim), 
                                        minval=-xavier_lim, maxval=xavier_lim, seed=22, dtype=tf.float32), dtype=tf.double)
        return weight_vals

    @tf.function
    def __call__(self, X):
        if not self.build:
            n_features = X.shape[1]
            self.weights = tf.Variable(self.xavier_init(shape=(n_features, self.neurons)), name="Weights", dtype=tf.double, trainable=True )
            self.bias = tf.Variable(tf.zeros(shape=1, dtype=tf.double), name="Bias", dtype=tf.double, trainable=True )
            self.build = True

        z = tf.add(tf.keras.layers.dot([X, self.weights], axes=0), self.bias)

        print(X.shape,"*",self.weights.shape,"+",self.bias.shape, "=", z.shape)
        return self.activation(z)

    

class MLPLinearRegressor(tf.Module):
    def __init__(self, layers, X, y, lr=0.001, epochs=10):
        self.layers = layers
        self.lr = lr
        self.epochs = epochs
        self.loaded = False
        self.X = X 
        self.XN = normalize_dataset(X)
        self.y = y
        self.optimizer = tf.keras.optimizers.Adam(lr)
        self.mse = tf.keras.metrics.MeanSquaredError()
        self.accuracy = tf.keras.metrics.MeanSquaredLogarithmicError()
        self.precision = tf.keras.metrics.Precision()
        self.loss_history = [e for e in range(epochs)]

    @tf.function(reduce_retracing=True)
    def accuracy_rrsse(self, predicted):
        return tf.divide(tf.sqrt(
            tf.divide(
                tf.reduce_sum(tf.square(tf.abs((tf.subtract(predicted, self.y))))),
                tf.reduce_sum(tf.square(tf.abs((tf.subtract(tf.reduce_mean(self.y), predicted)))))
            )), predicted.shape[0]
        )

    @tf.function(reduce_retracing=True)
    def loss(self, predicted):
        return tf.losses.mean_squared_error(self.y, predicted)

    @tf.function(reduce_retracing=True)
    def calc_metrics(self):
        self.acc_rrsse = self.accuracy_rrsse(self.predicted)
        self.accuracy.update_state(self.y, self.predicted)
        self.precision.update_state(self.y, self.predicted)
        self.mse.update_state(self.y, self.predicted)

    @tf.function(reduce_retracing=True)
    def train_step(self, X):
        for layer in self.layers:
            X = layer(X)
        return X

    def train(self):
        self.weights = model.train_step(self.XN)
        self.vars = [self.layers[0].weights, self.layers[1].weights, self.layers[2].weights,self.layers[3].weights,self.layers[4].weights, self.layers[0].bias, self.layers[1].bias, self.layers[2].bias,self.layers[3].bias,self.layers[4].bias]

        for e in range(self.epochs):
            with tf.GradientTape(watch_accessed_variables=True, persistent=True) as tape:
                self.weights = model.train_step(self.XN)
                self.predicted = tf.reduce_sum(tf.subtract(tf.keras.layers.dot([self.XN, tf.transpose(self.weights)], axes=0), self.layers[len(self.layers)-1].bias))
                loss = self.loss(self.predicted)
                self.loss_history[e] = loss.numpy()
                
            grads = tape.gradient(loss, self.vars)
            self.optimizer.apply_gradients(zip(grads, self.vars)) 
        print(self.predicted)

model = MLPLinearRegressor([
    Layer(neurons=10,),
    Layer(neurons=10,),
    Layer(neurons=10,),
    Layer(neurons=10,),
    Layer(neurons=X_train.shape[1], )
],X_train, tf.squeeze(y_train), lr=0.001, epochs=100)


model.train()
model.calc_metrics()


print("MeanSquaredError: ", model.mse.result().numpy())
print("Keras Accuracy: ", model.accuracy.result().numpy())
print("Keras Precision: ", model.precision.result().numpy())


fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].plot([e for e in range(model.epochs)], model.loss_history)

ax[1].scatter(X_train[:, 0], y_train[:])
ax[1].plot(X_train[:, 0], model.predicted[:])
plt.show()



ValueError: in user code:

    File "C:\Users\Utente\AppData\Local\Temp\ipykernel_21264\3358911384.py", line 69, in train_step  *
        X = layer(X)
    File "C:\Users\Utente\AppData\Local\Temp\ipykernel_21264\3358911384.py", line 24, in __call__  *
        z = tf.add(tf.keras.layers.dot([X, self.weights], axes=0), self.bias)
    File "c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\merging\dot.py", line 226, in dot  **
        return Dot(axes=axes, normalize=normalize, **kwargs)(inputs)
    File "c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\merging\dot.py", line 136, in build
        raise ValueError(

    ValueError: Incompatible input shapes: axis values 412 (at axis 0) != 8 (at axis 0). Full input shapes: (412